# Case Study: Automated Machine Learning (AutoML) for Autonomous Intrusion Detection System Development 
This is the code for the paper entitled "**[Enabling AutoML for Zero-Touch Network Security: Use-Case Driven Analysis](https://ieeexplore.ieee.org/document/10472316)**" published in *IEEE Transactions on Network and Service Management* (IF:5.3).<br>
Authors: Li Yang (liyanghart@gmail.com), Mirna El Rajab, Abdallah Shami, and Sami Muhaidat<br>

L. Yang, M. E. Rajab, A. Shami, and S. Muhaidat, "Enabling AutoML for Zero-Touch Network Security: Use-Case Driven Analysis," IEEE Transactions on Network and Service Management, pp. 1-28, 2024, doi: https://doi.org/10.1109/TNSM.2024.3376631.

# Code Part 3: Adversarial Machine Learning (AML) Attack and Defense
As many network services and functionalities rely on AI/ML models, they are becoming increasingly vulnerable to AML attacks. AML attacks exploit the weaknesses and vulnerabilities of ML models by generating adversarial inputs that can deceive or manipulate the models into making incorrect predictions. In networks, AML attacks pose a significant threat to overall network security and reliability.  
This case study aims to demonstrate the detrimental impact that AML attacks can have on ML models in networks and presents basic defense strategies to mitigate these attacks, thereby ensuring the accuracy of the ML-based IDS. In this case study, three common types of adversarial attacks ((i.e., DTA, FGSM, and BIM)) are used to generate adversarial samples to probe the vulnerability of the IDS.  
Subsequently, basic defense mechanisms, including adversarial sample detection and filtering, are devised to safeguard the AutoML-based IDS against AML attacks.  

## Dataset 2: 5G-NIDD
A subset of the network traffic data randomly sampled from the [5G-NIDD dataset](https://ieee-dataport.org/documents/5g-nidd-comprehensive-network-intrusion-detection-dataset-generated-over-5g-wireless).  

The 5G-NIDD dataset, created in December 2022, is a fully labeled resource constructed on a functional 5G test network for researchers and practitioners evaluating AI/ML solutions in the context of 5G/6G security [87]. 5G-NIDD encompasses data extracted from a 5G testbed connected to the 5G Test Network (5GTN) at the University of Oulu, Finland. The dataset is derived from two base stations, each featuring an attacker node and multiple benign 5G users. The attacker nodes target a server deployed within the 5GTN MEC environment. The attack scenarios captured in the dataset primarily include DoS attacks and port scans.

## Import libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score
import lightgbm as lgb
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.stats import shapiro
from imblearn.over_sampling import SMOTE
import time

In [2]:
import warnings 
warnings.filterwarnings('ignore')

## Read the sampled 5G-NIDD dataset

In [3]:
df = pd.read_csv("Data/5gnidd_0.01_pre-processed.csv")

In [4]:
df

,Seq,Dur,RunTime,Mean,Sum,Min,Max,Proto,sTos,dTos,...,SrcWin,DstWin,sVid,dVid,SrcTCPBase,DstTCPBase,TcpRtt,SynAck,AckDat,Label
0,0.000496,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.75,0.000000,0.000000,...,0.000002,0.00000,0.0,0.0,0.388654,0.000000,0.0,0.0,0.0,1
1,0.002442,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.75,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.388654,0.000000,0.0,0.0,0.0,1
2,0.003295,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.75,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.388654,0.000000,0.0,0.0,0.0,1
3,0.003317,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.75,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.388654,0.000000,0.0,0.0,0.0,1
4,0.003375,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.75,0.000000,0.000000,...,0.000000,0.00000,0.0,0.0,0.388669,0.000000,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12154,0.003688,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.00,0.000000,0.000000,...,0.000000,0.00000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0
12155,0.004104,2.000005e-07,2.000005e-07,2.000005e-07,2.000005e-07,2.000005e-07,2.000005e-07,0.75,0.000000,0.000000,...,0.000008,0.00025,0.0,0.0,0.292755,0.030151,0.0,0.0,0.0,0
12156,0.005234,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.00,0.000000,0.000000,...,0.000000,0.00000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0
12157,0.000015,4.079091e-02,4.079091e-02,4.079091e-02,4.079091e-02,4.079091e-02,4.079091e-02,0.50,0.830357,0.215054,...,0.000000,0.00000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0


# 1. Automated Data Pre-Processing

## Automated Transformation/Encoding
Automatically identify and transform string/text features into numerical features to make the data more readable by ML models

In [5]:
# Define the automated data encoding function
def Auto_Encoding(df):
    cat_features=[x for x in df.columns if df[x].dtype=="object"] ## Find string/text features
    le=LabelEncoder()
    for col in cat_features:
        if col in df.columns:
            i = df.columns.get_loc(col)
            # Transform to numerical features
            df.iloc[:,i] = df.apply(lambda i:le.fit_transform(i.astype(str)), axis=0, result_type='expand')
    return df

In [6]:
df=Auto_Encoding(df)

## Automated Imputation
Detect and impute missing values to improve data quality

In [7]:
# Define the automated data imputation function
def Auto_Imputation(df):
    if df.isnull().values.any() or np.isinf(df).values.any(): # if there is any empty or infinite values
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.fillna(0, inplace = True)  # Replace empty values with zeros; there are other imputation methods discussed in the paper
    return df

In [8]:
df=Auto_Imputation(df)

## Automated normalization
Normalize the range of features to a similar scale to improve data quality

In [9]:
def Auto_Normalization(df):
    stat, p = shapiro(df)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    # interpret
    alpha = 0.05
    numeric_features = df.drop(['Label'],axis = 1).dtypes[df.dtypes != 'object'].index
    
    # check if the data distribution follows a Gaussian/normal distribution
    # If so, select the Z-score normalization method; otherwise, select the min-max normalization
    # Details are in the paper
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
        df[numeric_features] = df[numeric_features].apply(
            lambda x: (x - x.mean()) / (x.std()))
        print('Z-score normalization is automatically chosen and used')
    else:
        print('Sample does not look Gaussian (reject H0)')
        df[numeric_features] = df[numeric_features].apply(
            lambda x: (x - x.min()) / (x.max()-x.min()))
        print('Min-max normalization is automatically chosen and used')
    return df

In [10]:
df=Auto_Normalization(df)

Statistics=0.563, p=0.000
Sample does not look Gaussian (reject H0)
Min-max normalization is automatically chosen and used


In [11]:
df=Auto_Imputation(df)

## Train-test split
Split the dataset into the training and the test set

In [12]:
X = df.drop(['Label'],axis=1)
y = df['Label']

# Here we used the 80%/20% split, it can be changed based on specific tasks
#X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, shuffle=False,random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2,random_state = 0)

## Automated data balancing
Generate minority class samples to solve class-imbalance and improve data quality.  
Synthetic Minority Over-sampling Technique (SMOTE) method is used.

In [13]:
pd.Series(y_train).value_counts()

1    5927
0    3800
Name: Label, dtype: int64

In [14]:
# For binary data (can be modified for multi-class data with the same logic)
def Auto_Balancing(X_train, y_train):
    number0 = pd.Series(y_train).value_counts().iloc[0]
    number1 = pd.Series(y_train).value_counts().iloc[1]
    
    if number0 > number1:
        nlarge = number0
    else:
        nlarge = number1
    
    # evaluate whether the incoming dataset is imbalanced (the abnormal/normal ratio is smaller than a threshold (e.g., 50%)) 
    if (number1/number0 > 1.5) or (number0/number1 > 1.5):
        smote=SMOTE(n_jobs=-1,sampling_strategy={0:nlarge, 1:nlarge})
        X_train, y_train = smote.fit_sample(X_train, y_train)
        
    return X_train, y_train

In [15]:
X_train, y_train = Auto_Balancing(X_train, y_train)

In [16]:
pd.Series(y_train).value_counts()

1    5927
0    5927
Name: Label, dtype: int64

# 2. Automated Feature Engineering
Feature selection method 1: **Information Gain (IG)**, used to remove irrelevant features to improve model efficiency  
Feature selection method 2: **Pearson Correlation**, used to remove redundant features to improve model efficiency and accuracy  

In [17]:
# Remove irrelevant features and select important features
def Feature_Importance_IG(data):
    features = data.drop(['Label'],axis=1).values  # "Label" should be changed to the target class variable name if different
    labels = data['Label'].values
    
    # Extract feature names
    feature_names = list(data.drop(['Label'],axis=1).columns)

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    model = lgb.LGBMRegressor(verbose = -1)
    model.fit(features, labels)
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})

    # Sort features according to importance
    feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index(drop = True)

    # Normalize the feature importances to add up to one
    feature_importances['normalized_importance'] = feature_importances['importance'] / feature_importances['importance'].sum()
    feature_importances['cumulative_importance'] = np.cumsum(feature_importances['normalized_importance'])
    
    cumulative_importance=0.90 # Only keep the important features with cumulative importance scores>=90%. It can be changed.

    # Make sure most important features are on top
    feature_importances = feature_importances.sort_values('cumulative_importance')

    # Identify the features not needed to reach the cumulative_importance
    record_low_importance = feature_importances[feature_importances['cumulative_importance'] > cumulative_importance]

    to_drop = list(record_low_importance['feature'])
#     print(feature_importances.drop(['importance'],axis=1))
    return to_drop

In [18]:
# Remove redundant features
def Feature_Redundancy_Pearson(data):
    correlation_threshold=0.90 # Only remove features with the redundancy>90%. It can be changed
    features = data.drop(['Label'],axis=1)
    corr_matrix = features.corr()

    # Extract the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(np.bool))

    # Select the features with correlations above the threshold
    # Need to use the absolute value
    to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

    # Dataframe to hold correlated pairs
    record_collinear = pd.DataFrame(columns = ['drop_feature', 'corr_feature', 'corr_value'])

    # Iterate through the columns to drop
    for column in to_drop:

        # Find the correlated features
        corr_features = list(upper.index[upper[column].abs() > correlation_threshold])

        # Find the correlated values
        corr_values = list(upper[column][upper[column].abs() > correlation_threshold])
        drop_features = [column for _ in range(len(corr_features))]    

        # Record the information (need a temp df for now)
        temp_df = pd.DataFrame.from_dict({'drop_feature': drop_features,
                                         'corr_feature': corr_features,
                                         'corr_value': corr_values})
        record_collinear = record_collinear.append(temp_df, ignore_index = True)
#     print(record_collinear)
    return to_drop

In [19]:
def Auto_Feature_Engineering(df):
    drop1 = Feature_Importance_IG(df)
    dfh1 = df.drop(columns = drop1)
    
    drop2 = Feature_Redundancy_Pearson(dfh1)
    dfh2 = dfh1.drop(columns = drop2)
    
    return dfh2

In [20]:
dfh2 = Auto_Feature_Engineering(df)
dfh2

,Seq,Dur,Proto,sTtl,dTtl,sHops,TotBytes,Offset,sMeanPktSz,dMeanPktSz,pLoss,Rate,SrcWin,DstWin,SrcTCPBase,TcpRtt,SynAck,Label
0,0.000496,0.000000e+00,0.75,0.211765,0.000000,0.357143,0.000007,0.000201,0.042208,0.000000,0.0,0.000000,0.000002,0.00000,0.388654,0.0,0.0,1
1,0.002442,0.000000e+00,0.75,0.145098,0.000000,0.964286,0.000007,0.000850,0.042208,0.000000,0.0,0.000000,0.000000,0.00000,0.388654,0.0,0.0,1
2,0.003295,0.000000e+00,0.75,0.149020,0.000000,0.928571,0.000007,0.001133,0.042208,0.000000,0.0,0.000000,0.000000,0.00000,0.388654,0.0,0.0,1
3,0.003317,0.000000e+00,0.75,0.211765,0.000000,0.357143,0.000007,0.001140,0.042208,0.000000,0.0,0.000000,0.000000,0.00000,0.388654,0.0,0.0,1
4,0.003375,0.000000e+00,0.75,0.211765,0.000000,0.357143,0.000007,0.001160,0.042208,0.000000,0.0,0.000000,0.000000,0.00000,0.388669,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12154,0.003688,0.000000e+00,1.00,0.976471,0.000000,0.250000,0.000014,0.003075,0.053852,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0
12155,0.004104,2.000005e-07,0.75,0.203922,0.250980,0.428571,0.000158,0.003462,0.039298,0.053791,0.0,1.000000,0.000008,0.00025,0.292755,0.0,0.0,0
12156,0.005234,0.000000e+00,1.00,0.976471,0.000000,0.250000,0.000014,0.004298,0.053852,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,0.0,0
12157,0.000015,4.079091e-02,0.50,1.000000,0.980392,0.035714,0.000145,0.004546,0.069862,0.060603,0.0,0.000003,0.000000,0.00000,0.000000,0.0,0.0,0


## Data Split & Balancing (After Feature Engineering)

In [21]:
X = dfh2.drop(['Label'],axis=1)
y = dfh2['Label']

#X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, shuffle=False,random_state = 0)
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2,random_state = 0)

In [22]:
X_train, y_train = Auto_Balancing(X_train, y_train)

# 3. Automated Model Selection
Select the best-performing model among five common machine learning models (Naive Bayes, KNN, random forest, LightGBM, and ANN/MLP) by evaluating their learning performance

## Model learning (for Comparison)

In [23]:
%%time
lg = lgb.LGBMClassifier(verbose = -1)
lg.fit(X_train,y_train)
t1=time.time()
predictions = lg.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 99.79400000000001%
Precision: 99.732%
Recall: 99.933%
F1-score: 99.832%
Time: 4.10164
Wall time: 322 ms


In [24]:
%%time
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 99.79400000000001%
Precision: 99.799%
Recall: 99.866%
F1-score: 99.832%
Time: 10.25318
Wall time: 753 ms


# 4. Hyperparameter Optimization
Optimize the best performing machine learning model (lightGBM) by tuning its hyperparameters

## Hold-out validation

In [25]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold

# Define the objective function
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']), 
        'max_depth': int(params['max_depth']),
        'learning_rate': abs(float(params['learning_rate'])),
        "num_leaves": int(params['num_leaves']),
        "min_child_samples": int(params['min_child_samples']),
    }
    clf = lgb.LGBMClassifier( **params)
    clf.fit(X_train,y_train)
    predictions = clf.predict(X_test)
    score = accuracy_score(y_test,predictions)
    return {'loss':-score, 'status': STATUS_OK }

# Define the hyperparameter configuration space
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 20),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    "learning_rate":hp.uniform('learning_rate', 0, 1),
    "num_leaves":hp.quniform('num_leaves',100,2000,100),
    "min_child_samples":hp.quniform('min_child_samples',10,50,5),
}

# Detect the optimal hyperparameter values
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50)
print("LightGBM: Hyperopt estimated optimum {}".format(best))



100%|██████████| 50/50 [00:22<00:00,  2.18trial/s, best loss: -0.998766447368421]
LightGBM: Hyperopt estimated optimum {'learning_rate': 0.2326246024181119, 'max_depth': 16.0, 'min_child_samples': 50.0, 'n_estimators': 200.0, 'num_leaves': 200.0}


In [26]:
%%time
clf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")

Accuracy: 99.87700000000001%
Precision: 99.799%
Recall: 100.0%
F1-score: 99.899%
Wall time: 199 ms


After hyperparameter optimization, the hold-out accuracy has been improved from 99.806% to 99.841%

In [27]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

# Define the objective function
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']), 
        'max_depth': int(params['max_depth']),
        'min_samples_split': int(params['min_samples_split']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'criterion': str(params['criterion'])
    }
    clf = RandomForestClassifier(**params)
    clf.fit(X_train,y_train)
    predictions = clf.predict(X_test)
    score = accuracy_score(y_test,predictions)
    return {'loss':-score, 'status': STATUS_OK }

# Define the hyperparameter configuration space
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 500, 20),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 11, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 11, 1),
    'criterion': hp.choice('criterion', ['gini', 'entropy'])
}

# Detect the optimal hyperparameter values
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("RandomForest: Hyperopt estimated optimum {}".format(best))


100%|██████████| 20/20 [00:39<00:00,  1.95s/trial, best loss: -0.9991776315789473]
RandomForest: Hyperopt estimated optimum {'criterion': 1, 'max_depth': 37.0, 'min_samples_leaf': 2.0, 'min_samples_split': 6.0, 'n_estimators': 200.0}


In [28]:
%%time
clf = RandomForestClassifier(max_depth=40, n_estimators = 60, min_samples_split = 4,
                         min_samples_leaf = 1, criterion = 'entropy')
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")

Accuracy: 99.836%
Precision: 99.866%
Recall: 99.866%
F1-score: 99.866%
Wall time: 466 ms


# AML

### Experiment 1
* 1. Original ML model for IDS
* 2. generate adversarial samples using DecisionTreeAttack or other attacks
* 3. test ML model under attack
* 4. develop adversarial sample detection model
* 5. remove adversarial samples from the training set
* 6. re-train IDS model

## DecisionTreeAttack

### Step 1: Original ML model for IDS

In [29]:
%%time
clf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")

Accuracy: 99.87700000000001%
Precision: 99.799%
Recall: 100.0%
F1-score: 99.899%
Wall time: 247 ms


### Step 2: generate adversarial samples using DecisionTreeAttack

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from matplotlib import pyplot as plt
import numpy as np

from art.attacks.evasion import DecisionTreeAttack
from art.estimators.classification import SklearnClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
from sklearn.preprocessing import OneHotEncoder
y_train1 = y_train.values.reshape(-1, 1)
y_train1_AML = OneHotEncoder().fit_transform(y_train1).toarray()

clf_art = SklearnClassifier(clf)
attack = DecisionTreeAttack(clf_art)

# Generate adversarial examples
x_adv = attack.generate(X_train.values,y_train1_AML)

Decision tree attack: 100%|██████████| 11854/11854 [00:03<00:00, 2980.75it/s]


### Step 3: test ML model under attack

In [31]:
%%time
#DTA
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(x_adv,y_train)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 62.458999999999996%
Precision: 68.947%
Recall: 70.383%
F1-score: 69.658%
Time: 2.87043
Wall time: 224 ms


### Step 4: develop adversarial sample detection model


In [32]:
X_new = np.concatenate([x_adv,X_train])
y1 = pd.Series(np.ones(y_train.shape[0]))
y2 = pd.Series(np.zeros(y_train.shape[0]))
y_new = np.concatenate([y1,y2])


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_new,y_new, train_size = 0.1, test_size = 0.9,random_state = 0)

In [33]:
%%time
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(X_train_d,y_train_d)
t1=time.time()
predictions2 = rf.predict(X_test_d)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test_d,predictions2),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test_d,predictions2),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test_d,predictions2),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test_d,predictions2),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test_d)*1000000,5)))

Accuracy: 99.822%
Precision: 99.72%
Recall: 99.925%
F1-score: 99.822%
Time: 1.35512
Wall time: 234 ms


### Step 5: remove adversarial samples from the training set

In [34]:
detection_results = rf.predict(X_new)
X_new1 = X_new
y_new1 = np.concatenate([y_train,y_train])

indices_to_remove = [i for i in range(len(detection_results)) if detection_results[i] == 1]
for i in reversed(indices_to_remove):
    X_new1 = np.delete(X_new1, i, axis=0)
    y_new1 = np.delete(y_new1, i)

### Step 6: re-train IDS model

In [35]:
%%time
#DTA
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(X_new1,y_new1)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 99.87700000000001%
Precision: 99.799%
Recall: 100.0%
F1-score: 99.899%
Time: 2.87053
Wall time: 180 ms


## FastGradientMethod

### Step 1: Original ML model for IDS

In [36]:
%%time
clf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")

Accuracy: 99.87700000000001%
Precision: 99.799%
Recall: 100.0%
F1-score: 99.899%
Wall time: 275 ms


### Step 2: generate adversarial samples using DecisionTreeAttack

In [38]:
from keras.layers import Input,Dense,Dropout,BatchNormalization,Activation
from art.estimators.classification import KerasClassifier
from keras import Model
import keras.backend as K
import keras.callbacks as kcallbacks
from keras import optimizers
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping

from art.attacks import evasion
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import SklearnClassifier

def ANN(optimizer = 'sgd',neurons=16,batch_size=1024,epochs=80,activation='relu',patience=8,loss='binary_crossentropy'):
    inputs=Input(shape=(X.shape[1],))
    x=Dense(1000)(inputs)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Dropout(0.3)(x)
    x=Dense(256)(inputs)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Dropout(0.25)(x)
    x=Dense(2,activation='softmax')(x)
    model=Model(inputs=inputs,outputs=x,name='base_nlp')
    model.compile(optimizer='adam',loss='categorical_crossentropy')
#     model.compile(optimizer=Adam(lr = 0.01),loss='categorical_crossentropy',metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    return model

# Create a KerasClassifier
classifier = KerasClassifier(model=ANN(), clip_values=(0, 1))

# Create the FastGradientMethod attack
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Generate adversarial examples
x_adv = attack.generate(X_train.values)

### Step 3: test ML model under attack

In [39]:
%%time
#FGM
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(x_adv,y_train)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 40.583999999999996%
Precision: 97.82600000000001%
Recall: 3.0220000000000002%
F1-score: 5.863%
Time: 5.74126
Wall time: 188 ms


### Step 4: develop adversarial sample detection model


In [40]:
X_new = np.concatenate([x_adv,X_train])
y1 = pd.Series(np.ones(y_train.shape[0]))
y2 = pd.Series(np.zeros(y_train.shape[0]))
y_new = np.concatenate([y1,y2])


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_new,y_new, train_size = 0.1, test_size = 0.9,random_state = 0)

In [41]:
%%time
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(X_train_d,y_train_d)
t1=time.time()
predictions2 = rf.predict(X_test_d)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test_d,predictions2),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test_d,predictions2),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test_d,predictions2),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test_d,predictions2),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test_d)*1000000,5)))

Accuracy: 99.663%
Precision: 99.775%
Recall: 99.55000000000001%
F1-score: 99.66199999999999%
Time: 1.02829
Wall time: 183 ms


### Step 5: remove adversarial samples from the training set

In [42]:
detection_results = rf.predict(X_new)
X_new1 = X_new
y_new1 = np.concatenate([y_train,y_train])

indices_to_remove = [i for i in range(len(detection_results)) if detection_results[i] == 1]
for i in reversed(indices_to_remove):
    X_new1 = np.delete(X_new1, i, axis=0)
    y_new1 = np.delete(y_new1, i)

### Step 6: re-train IDS model

In [43]:
%%time
#FGM
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(X_new1,y_new1)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 99.87700000000001%
Precision: 99.799%
Recall: 100.0%
F1-score: 99.899%
Time: 3.28335
Wall time: 404 ms


## BasicIterativeMethod

### Step 1: Original ML model for IDS

In [44]:
%%time
clf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")

Accuracy: 99.87700000000001%
Precision: 99.799%
Recall: 100.0%
F1-score: 99.899%
Wall time: 224 ms


### Step 2: generate adversarial samples using FastGradientMethod

In [45]:
from keras.layers import Input,Dense,Dropout,BatchNormalization,Activation
from art.estimators.classification import KerasClassifier
from keras import Model
import keras.backend as K
import keras.callbacks as kcallbacks
from keras import optimizers
from keras.optimizers import Adam

from keras.callbacks import EarlyStopping

from art.attacks import evasion
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import SklearnClassifier

def ANN(optimizer = 'sgd',neurons=16,batch_size=1024,epochs=80,activation='relu',patience=8,loss='binary_crossentropy'):
    inputs=Input(shape=(X.shape[1],))
    x=Dense(1000)(inputs)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Dropout(0.3)(x)
    x=Dense(256)(inputs)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    x=Dropout(0.25)(x)
    x=Dense(2,activation='softmax')(x)
    model=Model(inputs=inputs,outputs=x,name='base_nlp')
    model.compile(optimizer='adam',loss='categorical_crossentropy')
#     model.compile(optimizer=Adam(lr = 0.01),loss='categorical_crossentropy',metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, pd.get_dummies(y).values,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    return model

# Create a KerasClassifier
classifier = KerasClassifier(model=ANN(), clip_values=(0, 1))

# Create the FastGradientMethod attack
attack = evasion.BasicIterativeMethod(estimator=classifier, eps=0.1, eps_step=0.1, max_iter=200, batch_size=32, verbose = False)

# Generate adversarial examples
x_adv = attack.generate(X_train.values)

### Step 3: test ML model under attack

In [46]:
%%time
#BIM
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(x_adv,y_train)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 38.734%
Precision: 33.333%
Recall: 0.067%
F1-score: 0.134%
Time: 2.46036
Wall time: 205 ms


### Step 4: develop adversarial sample detection model


In [47]:
X_new = np.concatenate([x_adv,X_train])
y1 = pd.Series(np.ones(y_train.shape[0]))
y2 = pd.Series(np.zeros(y_train.shape[0]))
y_new = np.concatenate([y1,y2])


X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_new,y_new, train_size = 0.1, test_size = 0.9,random_state = 0)

In [48]:
%%time
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(X_train_d,y_train_d)
t1=time.time()
predictions2 = rf.predict(X_test_d)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test_d,predictions2),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test_d,predictions2),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test_d,predictions2),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test_d,predictions2),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test_d)*1000000,5)))

Accuracy: 99.634%
Precision: 99.579%
Recall: 99.691%
F1-score: 99.63499999999999%
Time: 0.88804
Wall time: 184 ms


### Step 5: remove adversarial samples from the training set

In [49]:
detection_results = rf.predict(X_new)
X_new1 = X_new
y_new1 = np.concatenate([y_train,y_train])

indices_to_remove = [i for i in range(len(detection_results)) if detection_results[i] == 1]
for i in reversed(indices_to_remove):
    X_new1 = np.delete(X_new1, i, axis=0)
    y_new1 = np.delete(y_new1, i)

### Step 6: re-train IDS model

In [50]:
%%time
#BIM
rf = lgb.LGBMClassifier(max_depth=21, learning_rate= 0.6251262379739384, n_estimators = 60, 
                         num_leaves = 1900, min_child_samples = 30)
rf.fit(X_new1,y_new1)
t1=time.time()
predictions = rf.predict(X_test)
t2=time.time()
print("Accuracy: "+str(round(accuracy_score(y_test,predictions),5)*100)+"%")
print("Precision: "+str(round(precision_score(y_test,predictions),5)*100)+"%")
print("Recall: "+str(round(recall_score(y_test,predictions),5)*100)+"%")
print("F1-score: "+str(round(f1_score(y_test,predictions),5)*100)+"%")
print("Time: "+str(round((t2-t1)/len(y_test)*1000000,5)))

Accuracy: 99.836%
Precision: 99.732%
Recall: 100.0%
F1-score: 99.866%
Time: 2.87122
Wall time: 195 ms
